In [1]:
import pandas as pd
from IPython.display import Markdown, display
from ipywidgets import interact
from ipywidgets.widgets import (
    Dropdown, SelectionSlider, Checkbox
)
from datetime import datetime
import cufflinks as cf

## Getting the data 

In [73]:
path_dict = dict(
    confirmed = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                 "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Confirmed.csv"),
    death = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
             "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Deaths.csv"),
    recovered = ("https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/"
                 "csse_covid_19_data/csse_covid_19_time_series/time_series_19-covid-Recovered.csv")
)

In [99]:
def set_index(df):
    """Set the index for the data frame using the date
    
    Args:
        df: Pandas data frame obtained from John Hopkins repo
    """
    # Set region, country, lat and long as index
    index = pd.MultiIndex.from_frame(df.iloc[:, :4])
    # Set the index and transpose
    df = df.iloc[:, 4:].set_index(index).T
    # Set date as index
    return df.set_index(pd.to_datetime(df.index, dayfirst=False))

df_dict = {
    type_: pd.read_csv(path).pipe(set_index)
    for type_, path in path_dict.items()
}

In [171]:
def create_covid_plot(df_input, title, out_file=None): 
    label_total = '<< Total >>'

    # Add a new column with the total number per day
    df_input[(None, label_total, None, None)] = (
        df_input.groupby(level=1, axis=1)
        .sum().sum(axis=1)
    )


    df = (
        df_input
        .groupby(level=1, axis=1).sum()
    )
    max_str_len = len(label_total)
    df.columns = [c if len(c) <= max_str_len 
                  else f"{c[:max_str_len]}..." 
                  for c in df.columns]

    plot_type = 'line' 

    # Plot the data
    fig = df.iplot(kind=plot_type, yTitle=title, title=title, theme='ggplot',
                   margin=dict(l=0, r=160, b=0, t=50, pad=0), online=True,
                   asFigure=True, mode='lines+markers', rangeslider=True)

    # Keep only the total selected in the legend
    for tr in fig.select_traces():
        if tr['name'] != label_total: 
            tr.update({'visible': 'legendonly'})

    if out_file is not None:
        fig.write_html(out_file)
    
    return fig
#     fig.show()

In [173]:
create_covid_plot(df_dict['confirmed'], '', 't.html')

In [169]:
df.iplot?

In [151]:
trs = list(fig.select_traces())

In [156]:
# trs[0].update({'group': 'confirmed_cum'})

In [143]:
plot_info = (('confirmed', 'confirmed cases'),
             ('death', 'deaths'),
             ('recovered', 'recovered cases'))

for k, type_ in plot_info:
    create_html_plot(df_dict[k], F'Number of {type_} (cumulative)', 
                     f'plot_{k}_cum.html')
    create_html_plot(df_dict[k], F'Number of {type_} (per day)', 
                     f'plot_{k}_daily.html')

<iframe id="igraph" scrolling="no" style="border:none;"
        seamless="seamless" src="plot.html" height="100%" width="100%">
</iframe>